In [1]:
import numpy as np
import pickle

def pickle_load(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

def load_data():
    # 学習データ
    x_train = pickle_load('../dataset/x_train.pkl')
    t_train = pickle_load('../dataset/t_train.pkl')
    tokenizer_en = np.load('../dataset/tokenizer_en.npy').item()
    tokenizer_ja = np.load('../dataset/tokenizer_ja.npy').item()
 
    # テストデータ
    x_test = pickle_load('../dataset/x_test.pkl')

    return (x_train, t_train, tokenizer_en, tokenizer_ja, x_test)

x_train, t_train, tokenizer_en, tokenizer_ja, x_test = load_data()

##%%writefile /root/userspace/chap08/materials/submission_code.py
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import csv

def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))
### レイヤー定義 ###
class Embedding:
    # WRITE ME
    def __init__(self, vocab_size, emb_dim, scale=0.08):
        self.V = tf.Variable(tf.random_normal([vocab_size, emb_dim], stddev=scale), name='V')

    def __call__(self, x):
        return tf.nn.embedding_lookup(self.V, x)
class LSTM:
    def __init__(self, hid_dim, seq_len, initial_state, return_state = False, return_sequences = False, hold_state = False, name = None):
        self.cell = tf.nn.rnn_cell.BasicLSTMCell(hid_dim)
        self.initial_state = initial_state
        self.seq_len = seq_len
        self.return_state = return_state
        self.return_sequences = return_sequences
        self.hold_state = hold_state
        self.name = name

    def __call__(self, x):
        with tf.variable_scope(self.name):
            outputs, state = tf.nn.dynamic_rnn(self.cell, x, self.seq_len, self.initial_state)
        
        if self.hold_state:
            self.initial_state = state
        
        if not self.return_sequences:
            outputs = state.h
            
        if not self.return_state:
            return outputs
        
        return outputs, state
    
# WRITE ME

### グラフ構築 ###
tf.reset_default_graph()

emb_dim = 256
hid_dim = 256
pad_index = 0
en_vocab_size = len(tokenizer_en.word_index) + 1
ja_vocab_size = len(tokenizer_ja.word_index) + 1
detokenizer_en = dict(map(reversed, tokenizer_en.word_index.items()))
detokenizer_ja = dict(map(reversed, tokenizer_ja.word_index.items()))
x = tf.placeholder(tf.int32, [None, None], name='x')
seq_len = tf.reduce_sum(tf.cast(tf.not_equal(x, pad_index), tf.int32), axis=1)
t = tf.placeholder(tf.int32, [None, None], name='t')
seq_len_t_in = tf.reduce_sum(tf.cast(tf.not_equal(t, pad_index), tf.int32), axis=1) - 1

t_out = tf.one_hot(t[:, 1:], depth=ja_vocab_size, dtype=tf.float32)
t_out = t_out * tf.expand_dims(tf.cast(tf.not_equal(t[:, 1:], pad_index), tf.float32), axis=-1)

initial_state = tf.nn.rnn_cell.LSTMStateTuple(tf.zeros([tf.shape(x)[0], hid_dim]), tf.zeros([tf.shape(x)[0], hid_dim]))

# Encoder
h_e = Embedding(en_vocab_size, emb_dim)(x)
_, encoded_state = LSTM(hid_dim, seq_len, initial_state, return_state=True, name='encoder_lstm')(h_e)

# Decoder
decoder = [
    Embedding(ja_vocab_size, emb_dim),
    LSTM(hid_dim, seq_len_t_in, encoded_state, return_sequences=True, name='decoder_lstm'),
    tf.layers.Dense(ja_vocab_size, tf.nn.softmax)
] # 生成時に再利用するためにモデルの各レイヤーを配列で確保

# Decoderに変数を通す
h_d = decoder[0](t)
h_d = decoder[1](h_d)
y = decoder[2](h_d)

cost = -tf.reduce_mean(tf.reduce_sum(t_out * tf_log(y[:, :-1]), axis=[1, 2]))

train = tf.train.AdamOptimizer().minimize(cost)

# WRITE ME

### データの準備 ###
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train)
x_train_lens = [len(com) for com in x_train]
sorted_train_indexes = sorted(range(len(x_train_lens)), key=lambda x: -x_train_lens[x])

x_train = [x_train[ind] for ind in sorted_train_indexes]
t_train = [t_train[ind] for ind in sorted_train_indexes]

### 学習 ###
#n_epochs = 1
#batch_size = 1
n_epochs = 1
batch_size = 1
n_batches = len(x_train) // batch_size
n_batches_valid = len(x_valid) // batch_size

sess = tf.Session()

sess.run(tf.global_variables_initializer())
for epoch in range(n_epochs):
    # train
    train_costs = []
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size

        x_train_batch = np.array(pad_sequences(x_train[start:end], padding='post', value=pad_index))
        t_train_batch = np.array(pad_sequences(t_train[start:end], padding='post', value=pad_index))

        _, train_cost = sess.run([train, cost], feed_dict={x: x_train_batch, t: t_train_batch})
        train_costs.append(train_cost)
    # valid
    #x_valid_pad = np.array(pad_sequences(x_valid, padding='post', value=pad_index))
    #t_valid_pad = np.array(pad_sequences(t_valid, padding='post', value=pad_index))

    #valid_cost = sess.run(cost, feed_dict={x: x_valid_pad, t: t_valid_pad})
    #print('EPOCH: %i, Training Cost: %.3f, Validation Cost: %.3f' % (epoch+1, np.mean(train_costs), valid_cost))
    for i in range(n_batches_valid):
        start = i * batch_size
        end = start + batch_size

        x_valid_pad = np.array(pad_sequences(x_valid[start:end], padding='post', value=pad_index)) # バッチ毎のPadding
        t_valid_pad = np.array(pad_sequences(t_valid[start:end], padding='post', value=pad_index))

        valid_cost = sess.run(cost, feed_dict={x: x_valid_pad, t: t_valid_pad})
        print('EPOCH: %i, Training Cost: %.3f, Validation Cost: %.3f' % (epoch+1, np.mean(train_costs), valid_cost))
# WRITE ME

### 生成用グラフ構築 ###
#max_len = tf.placeholder(tf.int32, name='max_len') # iterationの繰り返し回数の限度

#def cond():
    # WRITE ME

#def body():
    # WRITE ME
bos_eos = tf.placeholder(tf.int32, [2], name='bos_eos')
max_len = tf.placeholder(tf.int32, name='max_len') # iterationの繰り返し回数の限度

def cond(t, continue_flag, init_state, seq_last, seq):
    unfinished = tf.not_equal(tf.reduce_sum(tf.cast(continue_flag, tf.int32)), 0)
    return tf.logical_and(t < max_len, unfinished)

def body(t, prev_continue_flag, init_state, seq_last, seq):
    decoder[1].initial_state = init_state
    
    # Decoderの再構築
    h = decoder[0](tf.expand_dims(seq_last, axis=-1))
    h = decoder[1](h)
    y = decoder[2](h)
    
    seq_t = tf.reshape(tf.cast(tf.argmax(y, axis=2), tf.int32), shape=[-1])
    next_state = decoder[1].initial_state
    
    continue_flag = tf.logical_and(prev_continue_flag, tf.not_equal(seq_t, bos_eos[1])) # flagの更新

    return [t+1, continue_flag, next_state, seq_t, seq.write(t, seq_t)]

# WRITE ME
decoder[1].hold_state = True
decoder[1].seq_len = None

seq_0 = tf.ones([tf.shape(x)[0]], tf.int32)*bos_eos[0]

t_0 = tf.constant(1)
f_0 = tf.cast(tf.ones_like(seq_0), dtype=tf.bool) # バッチ内の各系列で</s>が出たかどうかの未了flag(0:出た, 1:出てない)
seq_array = tf.TensorArray(dtype=tf.int32, size=1, dynamic_size=True).write(0, seq_0)

*_, seq = tf.while_loop(cond, body, loop_vars=[t_0, f_0, encoded_state, seq_0, seq_array])

res = tf.transpose(seq.stack())

### 生成 ###
bos_id_ja, eos_id_ja = tokenizer_ja.texts_to_sequences(['<s> </s>'])[0]
#y_pred = sess.run(
#    # WRITE ME
#)
y_pred = sess.run(res, feed_dict={
    x: pad_sequences(x_test, padding='post', value=pad_index),
    bos_eos: np.array([bos_id_ja, eos_id_ja]),
    max_len: 100
})

### 出力 ###
def get_raw_contents(dataset, num, bos_id, eos_id):
    result = []
    for index in dataset[num]:
        if index == eos_id:
            break
            
        result.append(index)
        
        if index == bos_id:
            result = []
            
    return result

for num in range(1000):
    pred = get_raw_contents(y_pred, num, bos_id_ja, eos_id_ja)
    print('元の文:', ' '.join([detokenizer_en[com] for com in x_test[num][1:-1]]))
    print('生成文:', ' '.join([detokenizer_ja[com] for com in pred]))

output = [get_raw_contents(y_pred, i, bos_id_ja, eos_id_ja) for i in range(len(y_pred))]

with open('../dataset/submission_gen.csv', 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerows(output)
#name 'detokenizer_en' is not defined

C:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
EPOCH: 1, Training Cost: 34.894, Validation Cost: 21.328
EPOCH: 1, Training Cost: 34.894, Validation Cost: 23.566
EPOCH: 1, Training Cost: 34.894, Validation Cost: 57.197
EPOCH: 1, Training Cost: 34.894, Validation Cost: 43.334
EPOCH: 1, Training Cost: 34.894, Validation Cost: 32.127
EPOCH: 1, Training Cost: 34.894, Validation Cost: 24.124
EPOCH: 1, Training Cost: 34.894, Validation Cost: 23.332
EPOCH: 1, Training Cost: 34.894, Validation Cost: 45.960
EPOCH: 1, Training Cost: 34.894, Validation Cost: 28.625
EPOCH: 1, Training Cost: 34.894, Validation Cost: 26.903
EPOCH: 1, Training Cost: 34.894, Validation Cost: 26.010
EPOCH: 1, Training Cost: 34.894, Validation Cost: 55.836
EPOCH: 1, Training Cost: 34.894, Validation Cost: 39.471
EPOCH: 1, Training Cost: 34

EPOCH: 1, Training Cost: 34.894, Validation Cost: 25.900
EPOCH: 1, Training Cost: 34.894, Validation Cost: 15.487
EPOCH: 1, Training Cost: 34.894, Validation Cost: 42.166
EPOCH: 1, Training Cost: 34.894, Validation Cost: 27.303
EPOCH: 1, Training Cost: 34.894, Validation Cost: 19.738
EPOCH: 1, Training Cost: 34.894, Validation Cost: 30.998
EPOCH: 1, Training Cost: 34.894, Validation Cost: 43.288
EPOCH: 1, Training Cost: 34.894, Validation Cost: 43.219
EPOCH: 1, Training Cost: 34.894, Validation Cost: 13.852
EPOCH: 1, Training Cost: 34.894, Validation Cost: 34.053
EPOCH: 1, Training Cost: 34.894, Validation Cost: 44.526
EPOCH: 1, Training Cost: 34.894, Validation Cost: 23.504
EPOCH: 1, Training Cost: 34.894, Validation Cost: 10.799
EPOCH: 1, Training Cost: 34.894, Validation Cost: 31.831
EPOCH: 1, Training Cost: 34.894, Validation Cost: 16.177
EPOCH: 1, Training Cost: 34.894, Validation Cost: 30.697
EPOCH: 1, Training Cost: 34.894, Validation Cost: 33.162
EPOCH: 1, Training Cost: 34.894

EPOCH: 1, Training Cost: 34.894, Validation Cost: 39.765
EPOCH: 1, Training Cost: 34.894, Validation Cost: 43.835
EPOCH: 1, Training Cost: 34.894, Validation Cost: 24.484
EPOCH: 1, Training Cost: 34.894, Validation Cost: 23.704
EPOCH: 1, Training Cost: 34.894, Validation Cost: 38.022
EPOCH: 1, Training Cost: 34.894, Validation Cost: 20.921
EPOCH: 1, Training Cost: 34.894, Validation Cost: 15.395
EPOCH: 1, Training Cost: 34.894, Validation Cost: 36.667
EPOCH: 1, Training Cost: 34.894, Validation Cost: 32.597
EPOCH: 1, Training Cost: 34.894, Validation Cost: 22.360
EPOCH: 1, Training Cost: 34.894, Validation Cost: 31.706
EPOCH: 1, Training Cost: 34.894, Validation Cost: 29.547
EPOCH: 1, Training Cost: 34.894, Validation Cost: 34.557
EPOCH: 1, Training Cost: 34.894, Validation Cost: 41.224
EPOCH: 1, Training Cost: 34.894, Validation Cost: 25.030
EPOCH: 1, Training Cost: 34.894, Validation Cost: 24.284
EPOCH: 1, Training Cost: 34.894, Validation Cost: 39.228
EPOCH: 1, Training Cost: 34.894

EPOCH: 1, Training Cost: 34.894, Validation Cost: 9.461
EPOCH: 1, Training Cost: 34.894, Validation Cost: 30.089
EPOCH: 1, Training Cost: 34.894, Validation Cost: 51.133
EPOCH: 1, Training Cost: 34.894, Validation Cost: 35.211
EPOCH: 1, Training Cost: 34.894, Validation Cost: 44.766
EPOCH: 1, Training Cost: 34.894, Validation Cost: 18.522
EPOCH: 1, Training Cost: 34.894, Validation Cost: 57.152
EPOCH: 1, Training Cost: 34.894, Validation Cost: 23.353
EPOCH: 1, Training Cost: 34.894, Validation Cost: 38.047
EPOCH: 1, Training Cost: 34.894, Validation Cost: 79.081
EPOCH: 1, Training Cost: 34.894, Validation Cost: 30.526
EPOCH: 1, Training Cost: 34.894, Validation Cost: 12.192
EPOCH: 1, Training Cost: 34.894, Validation Cost: 32.520
EPOCH: 1, Training Cost: 34.894, Validation Cost: 22.836
EPOCH: 1, Training Cost: 34.894, Validation Cost: 22.014
EPOCH: 1, Training Cost: 34.894, Validation Cost: 39.935
EPOCH: 1, Training Cost: 34.894, Validation Cost: 58.261
EPOCH: 1, Training Cost: 34.894,

...

EPOCH: 1, Training Cost: 34.894, Validation Cost: 30.288
EPOCH: 1, Training Cost: 34.894, Validation Cost: 20.880
EPOCH: 1, Training Cost: 34.894, Validation Cost: 12.570
EPOCH: 1, Training Cost: 34.894, Validation Cost: 18.353
EPOCH: 1, Training Cost: 34.894, Validation Cost: 28.689
EPOCH: 1, Training Cost: 34.894, Validation Cost: 73.857
EPOCH: 1, Training Cost: 34.894, Validation Cost: 62.062
EPOCH: 1, Training Cost: 34.894, Validation Cost: 10.874
EPOCH: 1, Training Cost: 34.894, Validation Cost: 34.902
EPOCH: 1, Training Cost: 34.894, Validation Cost: 26.490
EPOCH: 1, Training Cost: 34.894, Validation Cost: 22.619
EPOCH: 1, Training Cost: 34.894, Validation Cost: 37.327
EPOCH: 1, Training Cost: 34.894, Validation Cost: 41.328
EPOCH: 1, Training Cost: 34.894, Validation Cost: 21.378
EPOCH: 1, Training Cost: 34.894, Validation Cost: 55.903
EPOCH: 1, Training Cost: 34.894, Validation Cost: 46.886
EPOCH: 1, Training Cost: 34.894, Validation Cost: 39.618
EPOCH: 1, Training Cost: 34.894

EPOCH: 1, Training Cost: 34.894, Validation Cost: 17.790
EPOCH: 1, Training Cost: 34.894, Validation Cost: 36.538
EPOCH: 1, Training Cost: 34.894, Validation Cost: 28.952
EPOCH: 1, Training Cost: 34.894, Validation Cost: 56.859
EPOCH: 1, Training Cost: 34.894, Validation Cost: 42.892
EPOCH: 1, Training Cost: 34.894, Validation Cost: 58.932
EPOCH: 1, Training Cost: 34.894, Validation Cost: 12.716
EPOCH: 1, Training Cost: 34.894, Validation Cost: 7.005
EPOCH: 1, Training Cost: 34.894, Validation Cost: 23.215
EPOCH: 1, Training Cost: 34.894, Validation Cost: 38.566
EPOCH: 1, Training Cost: 34.894, Validation Cost: 42.028
EPOCH: 1, Training Cost: 34.894, Validation Cost: 23.372
EPOCH: 1, Training Cost: 34.894, Validation Cost: 28.367
EPOCH: 1, Training Cost: 34.894, Validation Cost: 47.463
EPOCH: 1, Training Cost: 34.894, Validation Cost: 40.132
EPOCH: 1, Training Cost: 34.894, Validation Cost: 27.699
EPOCH: 1, Training Cost: 34.894, Validation Cost: 51.949
EPOCH: 1, Training Cost: 34.894,

EPOCH: 1, Training Cost: 34.894, Validation Cost: 19.974
EPOCH: 1, Training Cost: 34.894, Validation Cost: 7.445
EPOCH: 1, Training Cost: 34.894, Validation Cost: 28.504
EPOCH: 1, Training Cost: 34.894, Validation Cost: 37.440
EPOCH: 1, Training Cost: 34.894, Validation Cost: 46.868
EPOCH: 1, Training Cost: 34.894, Validation Cost: 52.541
EPOCH: 1, Training Cost: 34.894, Validation Cost: 11.178
EPOCH: 1, Training Cost: 34.894, Validation Cost: 57.698
EPOCH: 1, Training Cost: 34.894, Validation Cost: 27.549
EPOCH: 1, Training Cost: 34.894, Validation Cost: 10.236
EPOCH: 1, Training Cost: 34.894, Validation Cost: 33.757
EPOCH: 1, Training Cost: 34.894, Validation Cost: 28.581
EPOCH: 1, Training Cost: 34.894, Validation Cost: 33.267
EPOCH: 1, Training Cost: 34.894, Validation Cost: 39.377
EPOCH: 1, Training Cost: 34.894, Validation Cost: 33.369
EPOCH: 1, Training Cost: 34.894, Validation Cost: 56.263
EPOCH: 1, Training Cost: 34.894, Validation Cost: 27.013
EPOCH: 1, Training Cost: 34.894,

EPOCH: 1, Training Cost: 34.894, Validation Cost: 31.198
EPOCH: 1, Training Cost: 34.894, Validation Cost: 46.050
EPOCH: 1, Training Cost: 34.894, Validation Cost: 20.586
EPOCH: 1, Training Cost: 34.894, Validation Cost: 35.146
EPOCH: 1, Training Cost: 34.894, Validation Cost: 32.268
EPOCH: 1, Training Cost: 34.894, Validation Cost: 23.226
EPOCH: 1, Training Cost: 34.894, Validation Cost: 10.964
EPOCH: 1, Training Cost: 34.894, Validation Cost: 61.681
EPOCH: 1, Training Cost: 34.894, Validation Cost: 61.974
EPOCH: 1, Training Cost: 34.894, Validation Cost: 22.653
EPOCH: 1, Training Cost: 34.894, Validation Cost: 31.958
EPOCH: 1, Training Cost: 34.894, Validation Cost: 29.335
EPOCH: 1, Training Cost: 34.894, Validation Cost: 31.181
EPOCH: 1, Training Cost: 34.894, Validation Cost: 46.859
EPOCH: 1, Training Cost: 34.894, Validation Cost: 35.764
EPOCH: 1, Training Cost: 34.894, Validation Cost: 12.864
EPOCH: 1, Training Cost: 34.894, Validation Cost: 37.049
EPOCH: 1, Training Cost: 34.894

生成文: 彼 を 守 ら せ て は いけ な い 。
元の文: it is nice and warm today .
生成文: 今日 は 、 い う の で す 。
元の文: the dog was burnt to death .
生成文: その 犬 は 死 ぬ 。
元の文: we besought that he might give us the medicine .
生成文: 私 は 彼 を よく 知 っ て い る の を やめ た 。
元の文: i am accustomed to cold weather .
生成文: 私 は すぐ に い ま す 。
元の文: it is a fact that i don 't know her name .
生成文: 彼女 の 言 う こと は 本当 だ ろ う か 。
元の文: who is your favorite player ?
生成文: 誰 が あなた の で す か 。
元の文: can i borrow your pen for a few minutes ?
生成文: あなた は 電話 を し ま す か 。
元の文: the clock has stopped .
生成文: その 時計 は トニー に な っ た 。
元の文: she is a twin .
生成文: 彼女 は 走 る の で す 。
元の文: i want a lot more .
生成文: 私 は 、 忙し い 。
元の文: why don 't you ask mr white ?
生成文: あなた は ドア を 開け て い る の で す か 。
元の文: i 'll give you five minutes to work out this problem .
生成文: この 本 を 読 み ま す よ 。
元の文: i always catch a cold in the winter .
生成文: 私 は その 映画 を 見 た 。
元の文: you must study hard .
生成文: あなた は 勉強 し て い ま す 。
元の文: i 'll find one later .
生成文: 私 は 、 、 私 を 見 て い る 。
元の文: don 't tell lies .
生成文: ど